<a href="https://colab.research.google.com/github/mahhhdy/thesis/blob/main/Classification_with_sklearn_dencity_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


this post has been used </br>
https://github.com/NourozR/Stock-Price-Prediction-LSTM/blob/1829ad5c8b3ece7b0e9c16fd76cf2550142683b2/StockPricePrediction.py#L11

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Activation, Dropout
from keras.layers import LSTM , SimpleRNN
from keras.layers import BatchNormalization
from sklearn.metrics import mean_squared_error
from google.colab import drive
from google.colab import files
# from keras.layers.normalization import batch_normalization

from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

np.random.seed(7)

########## import dataset ##########
drive.mount('/content/drive')
path='/content/drive/MyDrive/DataSets/BTCPrediction/15XAUUSD.csv'
# dataset = pd.read_csv(path,usecols=[1,2,3,4])
dataset = pd.read_csv(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset = dataset.rename({'Time':'Date','|PD|':'PD'},axis=1)

In [3]:
dataset = dataset.reindex(index = dataset.index[::-1])
dataset = dataset.drop(columns=['Date','PD%','PD% / ATR'])

In [4]:
# dataset = dataset.head(30)

In [5]:
obs = np.arange(1, len(dataset) + 1, 1)
obs

array([   1,    2,    3, ..., 1998, 1999, 2000])

In [6]:
dataset['Average'] = dataset[['Open','Close','Low','High']].mean(axis=1)
# dataset

In [7]:
dataset['State'] =( dataset['Close'] > dataset['Open']).astype(int)
# dataset

In [8]:
# TAKING DIFFERENT INDICATORS FOR PREDICTION
# OHLC_avg = dataset[['Average']].to_numpy()
OHLC_avg = dataset.iloc[:,:dataset.shape[1]-1].to_numpy()
# HLC_avg = dataset[['High', 'Low', 'Close']].mean(axis = 1)
State = dataset[['State']].to_numpy()

In [9]:
# plt.figure(figsize=(18,6))
# plt.plot(obs, OHLC_avg, 'r', label = 'AveraOHLC_avgge')
# plt.plot(obs, close_val, 'g', label = 'Closing price')
# plt.legend(loc = 'upper right')
# plt.show()

In [10]:
OHLC_avg.shape,State.shape

((2000, 7), (2000, 1))

In [11]:
def new_dataset(dataset, target , step_size):
	data_X, data_Y = [], []
	for i in range(len(dataset)-step_size):
		a = dataset[i:(i+step_size), :]
		data_X.append(a)
		data_Y.append(target[i + step_size])
	return np.array(data_X), np.array(data_Y)

In [12]:
X, Y = new_dataset(OHLC_avg, State , 1)
X.shape , Y.shape

((1999, 1, 7), (1999, 1))

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
# X = np.reshape(X , (len(X),1)) 
# scaler = MinMaxScaler(feature_range=(0, 1))
# X = scaler.fit_transform(X)

In [15]:
len_train = int(len(X) * 0.8)
len_test = len(X) - len_train

trainX, testX = X[0:len_train,:], X[len_train:len(OHLC_avg),:]
trainY, testY = Y[0:len_train],Y[len_train:len(OHLC_avg)]



In [16]:
trainX.shape , testX.shape  , '----' , trainY.shape , testY.shape 

((1599, 1, 7), (400, 1, 7), '----', (1599, 1), (400, 1))

In [17]:
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# trainX = trainX.reshape(trainX.shape[0], 1, trainX.shape[1])
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# testX = testX.reshape(testX.shape[0], 1, testX.shape[1])

step_size = 1

trainX.shape , testX.shape

((1599, 1, 7), (400, 1, 7))

In [24]:
from sklearn.neural_network import MLPClassifie
clf = MLPClassifier(random_state=1, max_iter=300).fit(trainX.reshape(-1,7), trainY)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
clf.predict_proba(trainX.reshape(-1,7))

array([[4.73374189e-07, 9.99999527e-01],
       [4.87511638e-07, 9.99999512e-01],
       [4.45878407e-07, 9.99999554e-01],
       ...,
       [5.04177491e-07, 9.99999496e-01],
       [3.00624645e-07, 9.99999699e-01],
       [1.19047543e-06, 9.99998810e-01]])

In [26]:
clf.predict(testX.reshape(-1,7))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [27]:
clf.score(testX.reshape(-1,7), testY)

0.485

In [ ]:
# batchSize=1

# # LSTM MODEL
# model = Sequential()
# # model.add(LSTM(32, input_shape=(trainX[0].shape), return_sequences = True))
# # model.add(LSTM(16))
# # model.add(Dense(1))

# model.add(Dense(units = 10,activation="tanh"))
# model.add(Dense(units = 1))
# model.add(Activation('sigmoid'))
# model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy' , metrics=["accuracy"])
# model.fit(trainX.reshape(-1,7), trainY, epochs = 100, batch_size = batchSize, validation_split = 0.2)

In [ ]:
trainX.shape , trainY.shape , testX.shape , testY.shape

In [ ]:
# batchSize=1

# model = Sequential()
# model.add(BatchNormalization(input_shape = trainX[0].shape ))
# model.add(LSTM(units = 1))
# # model.add(Dropout(0.2))
# # model.add(LSTM(units = 50, return_sequences = True))
# # model.add(Dropout(0.2))
# # model.add(LSTM(units = 50, return_sequences = True))
# # model.add(Dropout(0.2))
# # model.add(LSTM(units = 5))
# # model.add(Dropout(0.2))
# model.add(Dense(units = 10,activation="tanh"))
# model.add(Dense(units = 1))
# model.add(Activation('sigmoid'))
# model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy' , metrics=["accuracy"])
# model.fit(trainX, trainY, epochs = 100, batch_size = batchSize, validation_split = 0.2)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
trainPredict.shape , trainY.shape

In [ ]:
trainY[:5] ,  trainPredict[:5]

In [ ]:
# DE-NORMALIZING FOR PLOTTING

# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])

# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])

In [ ]:
# TRAINING RMSE
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train RMSE: %.2f' % (trainScore))

In [ ]:


# df = pd.DataFrame([trainY[0], columns = ['Name'])

In [ ]:
# TEST RMSE
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test RMSE: %.2f' % (testScore))

In [ ]:
# CREATING SIMILAR DATASET TO PLOT TRAINING PREDICTIONS
trainPredictPlot = np.empty_like(OHLC_avg)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[step_size:len(trainPredict)+step_size, :] = trainPredict

In [ ]:
# CREATING SIMILAR DATASSET TO PLOT TEST PREDICTIONS
testPredictPlot = np.empty_like(OHLC_avg)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(step_size*2)+1:len(OHLC_avg)-1, :] = testPredict

In [ ]:
trainPredict.shape , testPredict.shape

In [ ]:
# DE-NORMALIZING MAIN DATASET 
OHLC_avg = scaler.inverse_transform(OHLC_avg)

In [ ]:
# PLOT OF MAIN OHLC VALUES, TRAIN PREDICTIONS AND TEST PREDICTIONS
plt.figure(figsize=(24, 10))
plt.plot(OHLC_avg, 'g', label = 'original dataset')
plt.plot(trainPredictPlot, 'r', label = 'training set')
plt.plot(testPredictPlot, 'b', label = 'predicted stock price/test set')
plt.legend(loc = 'upper left')
plt.xlabel('Time in Days')
plt.ylabel('OHLC Value of Apple Stocks')
plt.show()

In [ ]:
# PREDICT FUTURE VALUES
last_val = testPredict[-1]
last_val_scaled = last_val/last_val
next_val = model.predict(np.reshape(last_val_scaled, (1,1,1)))
# print "Last Day Value:", np.asscalar(last_val)
# print "Next Day Value:", np.asscalar(last_val*next_val)
# print np.append(last_val, next_val)

print("Last Day Value:", last_val)
print("Next Day Value:", last_val*next_val)
# print(np.append(last_val, next_val)

In [ ]:
df2 = pd.DataFrame()

In [ ]:
trainY = trainY.T
testY = testY.T

In [ ]:
perdictCol = np.vstack((trainPredict , testPredict))  
dataCol = np.vstack((trainY , testY))  

perdictCol  = perdictCol.reshape(-1)
dataCol  = dataCol.reshape(-1)

perdictCol.shape , dataCol.shape

In [ ]:
my_array = np.array([perdictCol,dataCol]).T

In [ ]:
df2 = pd.DataFrame(my_array, columns = ['perdictCol','dataCol'])
df2['diff'] = df2['perdictCol'] - df2['dataCol']

In [ ]:
df2.tail()

In [ ]:
(df2['diff']>2).sum() + (df2['diff'] < -2).sum() 

In [ ]:
distance, warp_path = fastdtw(df2['perdictCol'],df2['dataCol'], dist=euclidean)

In [ ]:
distance

DTW Plot

In [ ]:
# fig, ax = plt.subplots(figsize=(16, 12))

# # Remove the border and axes ticks
# fig.patch.set_visible(False)
# ax.axis('off')

# for [map_x, map_y] in warp_path:
#     ax.plot([map_x, map_y], [df2['perdictCol'][map_x], df2['dataCol'][map_y]], '-k')

# ax.plot(df2['perdictCol'], color='blue', marker='o', markersize=10, linewidth=5)
# ax.plot(df2['dataCol'], color='red', marker='o', markersize=10, linewidth=5)
# ax.tick_params(axis="both", which="major", labelsize=18)

# # fig.savefig("ex2_dtw_distance.png", **savefig_options)

# KMeans on OHLC_avg for clusetering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(4)
kmeans.fit(OHLC_avg)

In [ ]:
identified_clusters = kmeans.fit_predict(OHLC_avg)
identified_clusters

In [ ]:
(identified_clusters==0).sum()

In [ ]:
OHLC_avg.shape

# sklearn density


In [ ]:
OHLC_avg = OHLC_avg.flatten()
OHLC_avg.shape

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
y_d = np.linspace(OHLC_avg.min(), OHLC_avg.max(), 2000)

In [ ]:
# instantiate and fit the KDE model
kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
kde.fit([OHLC_avg])

# score_samples returns the log of the probability density
logprob = kde.score_samples([OHLC_avg])

plt.fill_between(y_d, np.exp(logprob), alpha=0.5)
plt.plot(np.full_like(OHLC_avg, -0.01), OHLC_avg , '|k', markeredgewidth=1)
plt.ylim(OHLC_avg.min(), OHLC_avg.max())


In [ ]:
x_d = np.linspace(OHLC_avg.min(), OHLC_avg.max(), 2000)

In [ ]:
# instantiate and fit the KDE model
kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
kde.fit([OHLC_avg])

# score_samples returns the log of the probability density
logprob = kde.score_samples([OHLC_avg])

plt.fill_between(x_d,np.exp(logprob) , alpha=0.5)
plt.plot(OHLC_avg, np.full_like(OHLC_avg, -0.01), '|k', markeredgewidth=1)
plt.hist(OHLC_avg, bins=60)
plt.xlim(OHLC_avg.min(), OHLC_avg.max())

In [ ]:
# def make_data(N, f=0.3, rseed=1):
#     rand = np.random.RandomState(rseed)
#     x = rand.randn(N)
#     x[int(f * N):] += 5
#     return x

x = OHLC_avg

In [ ]:
from scipy.stats import norm
x_d = np.linspace(OHLC_avg.min(), OHLC_avg.max(), 2000)
x_ticks = np.around(np.linspace(OHLC_avg.min(), OHLC_avg.max(), 20),4)

In [ ]:
x_ticks

In [ ]:
from matplotlib.ticker import FormatStrFormatter


In [ ]:
from sklearn.neighbors import KernelDensity

kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
kde.fit(x[:, None])

logprob = kde.score_samples(x_d[:, None])

plt.figure(figsize=(25,6))
plt.fill_between(x_d, np.exp(logprob), alpha=0.5)
plt.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
# plt.ylim(-0.02, 0.22)

plt.xticks(x_ticks)
plt.xticks(rotation=90)
